In [13]:
import pandas as pd
import numpy as np
import os

iteration = 1

while True:
    print(f"Ejecutando iteración {iteration}...")

    df_courses_vrai_file = f'formulario_limpio_{iteration}.xlsx'

    if not os.path.exists(df_courses_vrai_file):
        print(f"Archivo {df_courses_vrai_file} no encontrado. Fin.")
        break

    cols_df_students = ['Nombre completo', 'Transcript of Records', 'RUT UC', 'Correo electrónico2','País de tu universidad', 'Universidad de Origen', 'NRC', 'Sigla', 'Unidad Académica', 'Convenio específico', 'Curso dentro del catálogo', 'Prioridad', 'Nivel', 'UA convenio', 'Convenio específico', '¿Necesitas este curso para la obtención del grado en tu universidad de origen?']
    df_students = pd.read_excel(df_courses_vrai_file, sheet_name='Cursos validados', dtype=str)[cols_df_students].astype(str)
    names_ruts = df_students[['Nombre completo', 'RUT UC']].drop_duplicates(subset=['RUT UC'])
    df_students = df_students[df_students['Curso dentro del catálogo'] == 'Sí']

    cols_df_courses = ['NRC', 'Escuela', 'Nombre Curso', 'Sigla', 'Sección', 'Campus', 'Horario Cátedra/Clase', 'Horario Ayudantía', 'Horario Laboratorio', 'Horario Taller', 'Vacantes Ofrecidas Máximas', 'Nivel']
    df_courses = pd.read_excel(df_courses_vrai_file, sheet_name='Catálogo VRAI', dtype=str)[cols_df_courses].astype(str)

    cols_df_convenios = ['País', 'Institución', 'Nivel', 'Área', 'Estado Actual']
    df_convenios = pd.read_excel(df_courses_vrai_file, sheet_name='Convenios', dtype=str)[cols_df_convenios].astype(str)
    df_convenios = df_convenios[df_convenios['Estado Actual'].isin(['Vigente', 'Renovación Automática', 'Duración indefinida'])]
    ### convenios
    df_students['convenio activo'] = False
    for i, row in df_students.iterrows():
        uni = row['Universidad de Origen']
        ua_student = row['UA convenio']
        ua_course = row['Unidad Académica']
        nrc = row['NRC']
        
        df_students.loc[df_students['UA convenio'] == 'General', 'convenio activo'] = True
        df_students.loc[df_students['Unidad Académica'] == df_students['UA convenio'], 'convenio activo'] = True

    ###
    priority_values = {1 : 150, 2 : 130, 3 : 110, 4 : 100, 5 : 70, 6 : 60, 7 : 50, 8 : 40, 9 : 30, 10 : 20}
    start_times_info = { '0820' : 1, '0940' : 2, '1100' : 3, '1220' : 4, '1450' : 5, '1610' : 6, '1730' : 7, '1850' : 8, '2010' : 9}
    end_times_info = { '0930' : 1, '1050' : 2, '1210' : 3, '1330' : 4, '1600' : 5, '1720' : 6, '1840' : 7, '2000' : 8, '2120' : 9}

    def format_schedule_info(info):
        return info.strip().replace(' ', '').replace(':', '').replace('\n', '')[:-1].split(';')

    def generate_priority_combinations(n_courses):
        all_combinations = []
        for i in range(1, n_courses + 1):
            for j in range(i + 1, n_courses + 1):
                all_combinations.append([[i, j], i + j])
        all_combinations.sort(key = lambda x : x[1])
        return all_combinations

    def transform_module_info(schedule1:dict, full_schedule_info:list):
        for info in full_schedule_info:
            info_list = format_schedule_info(info)
            for element in info_list:
                week_day, start_time, end_time = element[-10], element[-9:-5], element[-4:]
                start_module, end_module = start_times_info[start_time], end_times_info[end_time]
                for i in range(start_module, end_module + 1):
                    schedule1[week_day].append(i)
        return schedule1

    def get_full_schedule(course_match):
        campus = course_match['Campus'].values[0].strip()
        catedra_schedule, ayudantia_schedule = course_match['Horario Cátedra/Clase'].values[0], course_match['Horario Ayudantía'].values[0]
        taller_schedule, lab_schedule = course_match['Horario Taller'].values[0], course_match['Horario Laboratorio'].values[0]
        full_schedule_list = [x for x in [catedra_schedule, ayudantia_schedule, taller_schedule, lab_schedule] if x != 'nan']
        return full_schedule_list, campus

    def get_class1_info(course_match): # Extraer la información de los horarios del curso 1, convertirla y transferirla al diccionario schedule1.
        schedule1 = {'L': [], 'M' : [], 'W' : [], 'J' : [], 'V' : [], 'S' : []}
        full_schedule_info, campus1 = get_full_schedule(course_match)
        schedule1 = transform_module_info(schedule1, full_schedule_info)
        return schedule1, campus1

    def check_class2_compatibility(schedule1, campus1, course_match):
        full_schedule_info, campus2 = get_full_schedule(course_match)
        for info in full_schedule_info:
            info_list = format_schedule_info(info)
            for element in info_list:
                week_day, start_time, end_time = element[-10], element[-9:-5], element[-4:]
                start_module, end_module = start_times_info[start_time], end_times_info[end_time]
                for i in range(start_module, end_module + 1):
                    if i in schedule1[week_day] or (campus1 != campus2 and (i + 1 in schedule1[week_day] or i - 1 in schedule1[week_day])):
                        return False
        return True

    def check_compatibility(nrc1, nrc2, sigla1, sigla2):
        course1_match = df_courses.loc[df_courses['NRC'] == nrc1]
        course2_match = df_courses.loc[df_courses['NRC'] == nrc2]
        if len(course1_match) and len(course2_match) and sigla1 != sigla2:
            schedule1, campus1 = get_class1_info(course1_match)
            compatible = check_class2_compatibility(schedule1, campus1, course2_match)
            return compatible, [course1_match, course2_match]
        return False, []

    # ['Nombre completo', 'Transcript of Records', 'RUT UC', 'Correo electrónico2',
    # 'País de tu universidad', 'Universidad de Origen', 'NRC', 'Sigla', 'Curso dentro del catálogo']
    #Generar todas las posibles combinaciones de cursos compatibles, ordenadas por prioridad
    compatible_list = []
    for index, row in names_ruts.iterrows():
        compatible = False
        matches = df_students.loc[(df_students['RUT UC'] == row['RUT UC']) & (df_students['Curso dentro del catálogo'] == 'Sí')]
        if len(matches) >= 2:
            student_info = { 'name': matches['Nombre completo'].values[0], 'tor': matches['Transcript of Records'].values[0], 
                        'rut_uc': matches['RUT UC'].values[0], 'email': matches['Correo electrónico2'].values[0], 
                        'uni_country': matches['País de tu universidad'].values[0], 
                        'uni_name' : matches['Universidad de Origen'].values[0] }
            all_priority_combinations = generate_priority_combinations(len(matches))
            for priority_combination in all_priority_combinations:
                index_course1, index_course2 = priority_combination[0][0], priority_combination[0][1] 
                nrc_course1, nrc_course2 = matches['NRC'].values[index_course1 - 1], matches['NRC'].values[index_course2 - 1]
                sigla_course1, sigla_course2 = matches['Sigla'].values[index_course1 - 1], matches['Sigla'].values[index_course2 - 1]
                ua_course1, ua_course2 = matches['Unidad Académica'].values[index_course1 - 1], matches['Unidad Académica'].values[index_course2 - 1]
                priority_course1, priority_course2 = matches['Prioridad'].values[index_course1 - 1], matches['Prioridad'].values[index_course2 - 1]
                convenio_course1, convenio_course2 = matches['convenio activo'].values[index_course1 - 1], matches['convenio activo'].values[index_course2 - 1]
                reqgrad_course1, reqgrad_course2 =  matches['¿Necesitas este curso para la obtención del grado en tu universidad de origen?'].values[index_course1 - 1], matches['¿Necesitas este curso para la obtención del grado en tu universidad de origen?'].values[index_course2 - 1]
                compatible, courses = check_compatibility(nrc_course1, nrc_course2, sigla_course1, sigla_course2)
                if compatible:
                    compatible_list.append({
                    'Nombre': student_info['name'],
                    'RUT UC': student_info['rut_uc'],
                    'Email': student_info['email'],
                    'Curso 1 - NRC': nrc_course1,
                    'Curso 1 - Sigla': sigla_course1,
                    'Curso 1 - Unidad Académica': ua_course1,
                    'Curso 1 - Prioridad': priority_course1,
                    'Curso 1 - Convenio': convenio_course1,
                    'Curso 1 - ReqGrad': reqgrad_course1,
                    'Curso 2 - NRC': nrc_course2,
                    'Curso 2 - Sigla': sigla_course2,
                    'Curso 2 - Unidad Académica': ua_course2,
                    'Curso 2 - Prioridad': priority_course2,
                    'Curso 2 - Convenio': convenio_course2,
                    'Curso 2 - ReqGrad': reqgrad_course2,
                    })
    #Almacenar como dataframe y definir prioridad
    df_compatible = pd.DataFrame(compatible_list)
    df_compatible['Curso 1 - Prioridad'] = df_compatible['Curso 1 - Prioridad'].astype(int)
    df_compatible['Curso 2 - Prioridad'] = df_compatible['Curso 2 - Prioridad'].astype(int)
    df_compatible['Score 1'] = df_compatible['Curso 1 - Prioridad'].map(priority_values)
    df_compatible['Score 2'] = df_compatible['Curso 2 - Prioridad'].map(priority_values)
    df_compatible['Total Score'] = df_compatible['Score 1'] + df_compatible['Score 2']
    df_compatible = df_compatible.sort_values(by=['Nombre', 'Total Score'], ascending=[True, False])

    #Almacenar combinacion preferida por cada estudiante dentro de las combinaciones viables
    df_preferred = pd.DataFrame(df_compatible).drop_duplicates(subset='Nombre', keep='first')

    #Generar dataframe con demanda de cursos
    df_course_demand = pd.DataFrame(columns=['RUT UC', 'Nombre completo', 'NRC', 'Unidad Académica', 'Prioridad', 'Convenio', 'ReqGrad', 'Curso_num'])

    for i in [1, 2]:
        temp_df = df_preferred[['RUT UC', 'Nombre',
                                f'Curso {i} - NRC',
                                f'Curso {i} - Unidad Académica',
                                f'Curso {i} - Prioridad',
                                f'Curso {i} - Convenio',
                                f'Curso {i} - ReqGrad']].copy()
        
        temp_df.columns = ['RUT UC', 'Nombre completo', 'NRC', 'Unidad Académica', 'Prioridad', 'Convenio', 'ReqGrad']
        temp_df['Curso_num'] = i
        df_course_demand = pd.concat([df_course_demand, temp_df], ignore_index=True)

    #Verificar cupos disponibles
    df_course_demand = df_course_demand.merge(
        df_courses[['NRC', 'Vacantes Ofrecidas Máximas','Nombre Curso','Sigla']],
        on='NRC',
        how='left'
    )
    #Convertir cupos a valor numerico
    df_course_demand['Vacantes Ofrecidas Máximas'] = pd.to_numeric(df_course_demand['Vacantes Ofrecidas Máximas'], errors='coerce')

    #Generar dataframe de demanda de cursos (ordenada por prioridad)
    df_course_demand['random_order'] = np.random.rand(len(df_course_demand))

    df_course_demand_sorted = df_course_demand.sort_values(
        by=['NRC', 'Prioridad', 'Convenio', 'ReqGrad', 'random_order'],
        ascending=[True, False, False, True, True]
    )

    #Asignar cursos a personas con mayor prioridad
    assigned_nrc_counts = {}
    assigned_students = []

    for _, row in df_course_demand_sorted.iterrows():
        nrc = row['NRC']
        vacantes = row['Vacantes Ofrecidas Máximas']
        
        if pd.isna(vacantes):
            continue  # skip if vacancies unknown

        assigned = assigned_nrc_counts.get(nrc, 0)
        
        if assigned < vacantes:
            assigned_nrc_counts[nrc] = assigned + 1
            assigned_students.append(row)
    df_matriculados = pd.DataFrame(assigned_students)


    #Detectar personas sin cursos asignados
    ruts_counts = df_matriculados['RUT UC'].value_counts()
    df_no_matriculados = ruts_counts[ruts_counts < 2].index.tolist()
    df_matriculados = df_matriculados[df_matriculados['RUT UC'].isin(ruts_counts[ruts_counts >= 2].index)]

    all_ruts = set(df_students['RUT UC'].unique())
    matriculated_ruts = set(df_matriculados['RUT UC'].unique())
    non_matriculated_ruts = all_ruts - matriculated_ruts
    df_no_matriculados = list(set(df_no_matriculados) | non_matriculated_ruts)
    df_no_matriculados = df_students[df_students['RUT UC'].isin(df_no_matriculados)][['Nombre completo', 'RUT UC']].drop_duplicates()

    with pd.ExcelWriter(f'output_{iteration}.xlsx') as writer:
        df_matriculados.to_excel(writer, sheet_name='Con éxito', index=False)
        df_no_matriculados.to_excel(writer, sheet_name='Sin éxito', index=False)
        df_compatible.to_excel(writer, sheet_name='Posibles combinaciones', index=False)
    print(f' output_{iteration}.xlsx guardado con {len(df_matriculados)} estudiantes asignados.')
    print(f'Cantidad de combinaciones compatibles encontradas: {len(df_compatible)}')
    print(f'Estudiantes con combinaciones viables: {df_compatible["Nombre"].nunique()}')

    df_cursos_validados = pd.read_excel(df_courses_vrai_file, sheet_name='Cursos validados', dtype=str)
    df_catalogo = pd.read_excel(df_courses_vrai_file, sheet_name='Catálogo VRAI', dtype=str)
    df_convenio = pd.read_excel(df_courses_vrai_file, sheet_name='Convenios', dtype=str)

    df_cursos_validados_filtrado = df_cursos_validados[df_cursos_validados['RUT UC'].isin(df_no_matriculados['RUT UC'])]

    df_catalogo['Vacantes Ofrecidas Máximas'] = pd.to_numeric(df_catalogo['Vacantes Ofrecidas Máximas'], errors='coerce')
    asignados_por_nrc = df_matriculados['NRC'].value_counts()

    df_catalogo['Vacantes Ofrecidas Máximas'] = df_catalogo.apply(
        lambda row: row['Vacantes Ofrecidas Máximas'] - asignados_por_nrc.get(row['NRC'], 0)
        if pd.notna(row['Vacantes Ofrecidas Máximas']) else row['Vacantes Ofrecidas Máximas'],
        axis=1
    )

    # Guardar nuevo archivo para la siguiente iteración
    with pd.ExcelWriter(f'formulario_limpio_{iteration+1}.xlsx', engine='openpyxl') as writer:
        df_cursos_validados_filtrado.to_excel(writer, sheet_name='Cursos validados', index=False)
        df_catalogo.to_excel(writer, sheet_name='Catálogo VRAI', index=False)
        df_convenio.to_excel(writer, sheet_name='Convenios', index=False)

    if df_matriculados.empty:   
        print("No hay más asignaciones exitosas.")
        break

    iteration += 1   

    # Guardemos info para saber sobre las vacantes VRAI

    if not os.path.exists('formulario_limpio_1.xlsx'):
        print("No se encontró el archivo formulario_limpio_1.xlsx para extraer las vacantes originales.")
    else:
        df_ini = pd.read_excel('formulario_limpio_1.xlsx', sheet_name='Catálogo VRAI', dtype=str)
        df_fin = pd.read_excel(f'formulario_limpio_{iteration}.xlsx', sheet_name='Catálogo VRAI', dtype=str)

        vacantes_iniciales = pd.to_numeric(df_ini['Vacantes Ofrecidas Máximas'], errors='coerce')
        vacantes_finales = pd.to_numeric(df_fin['Vacantes Ofrecidas Máximas'], errors='coerce')

        df_fin['Vacantes Iniciales'] = vacantes_iniciales
        df_fin['Vacantes Usadas'] = vacantes_iniciales - vacantes_finales
        df_fin['Vacantes Restantes'] = vacantes_finales

        with pd.ExcelWriter(f'formulario_limpio_{iteration}.xlsx', engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
            df_fin.to_excel(writer, sheet_name='Catálogo VRAI', index=False)






Ejecutando iteración 1...
 output_1.xlsx guardado con 496 estudiantes asignados.
Cantidad de combinaciones compatibles encontradas: 4947
Estudiantes con combinaciones viables: 369
Ejecutando iteración 2...
 output_2.xlsx guardado con 4 estudiantes asignados.
Cantidad de combinaciones compatibles encontradas: 1617
Estudiantes con combinaciones viables: 121
Ejecutando iteración 3...
 output_3.xlsx guardado con 2 estudiantes asignados.
Cantidad de combinaciones compatibles encontradas: 1603
Estudiantes con combinaciones viables: 119
Ejecutando iteración 4...
 output_4.xlsx guardado con 2 estudiantes asignados.
Cantidad de combinaciones compatibles encontradas: 1577
Estudiantes con combinaciones viables: 118
Ejecutando iteración 5...
 output_5.xlsx guardado con 0 estudiantes asignados.
Cantidad de combinaciones compatibles encontradas: 1571
Estudiantes con combinaciones viables: 117
No hay más asignaciones exitosas.


# Ahora vamos con asignar UN CURSO


In [14]:
# Detectar el último output generado en la fase de pares (el codigo de arriba)
iteration = 1
while os.path.exists(f'output_{iteration}.xlsx'):
    iteration += 1
iteration -= 1  # último exitoso
print(f"Iniciando asignación individual desde output_{iteration}.xlsx...")

# Archivos base
output_file = f'output_{iteration}.xlsx'
formulario_file = f'formulario_limpio_{iteration + 1}.xlsx'  # siguiente al último formulario generado

# Cargar estudiantes sin éxito
df_no_exito = pd.read_excel(output_file, sheet_name='Sin éxito', dtype=str)
ruts_no_exito = df_no_exito['RUT UC'].unique()

# Cargar cursos validados y catálogo
df_validados = pd.read_excel(formulario_file, sheet_name='Cursos validados', dtype=str)
df_catalogo = pd.read_excel(formulario_file, sheet_name='Catálogo VRAI', dtype=str)
df_convenios = pd.read_excel(formulario_file, sheet_name='Convenios', dtype=str)
df_catalogo['Vacantes Ofrecidas Máximas'] = pd.to_numeric(df_catalogo['Vacantes Ofrecidas Máximas'], errors='coerce')

# Filtrar cursos validados y convenios activos
df_validados = df_validados[df_validados['Curso dentro del catálogo'] == 'Sí']
df_validados = df_validados[df_validados['RUT UC'].isin(ruts_no_exito)]
df_convenios = df_convenios[df_convenios['Estado Actual'].isin(['Vigente', 'Renovación Automática', 'Duración indefinida'])]

# Agregar columna de convenio activo
df_validados['convenio activo'] = False

for i, row in df_validados.iterrows():
    uni = row['Universidad de Origen']
    ua_student = row['UA convenio']
    ua_course = row['Unidad Académica']
    nrc = row['NRC']
    
    df_students.loc[df_students['UA convenio'] == 'General', 'convenio activo'] = True
    df_students.loc[df_students['Unidad Académica'] == df_students['UA convenio'], 'convenio activo'] = True

# Asignación individual
single_iter = 1

while True:
    print(f"Iteración de asignación individual {single_iter}")
    df_demand = df_validados.copy()

    # Ordenar por prioridad, convenio, necesidad de grado, y aleatorio
    df_demand['Prioridad'] = pd.to_numeric(df_demand['Prioridad'], errors='coerce')
    df_demand['random'] = np.random.rand(len(df_demand))
    df_demand_sorted = df_demand.sort_values(
        by=[
            'RUT UC',
            'Prioridad',
            'convenio activo',
            '¿Necesitas este curso para la obtención del grado en tu universidad de origen?',
            'random'
        ],
        ascending=[True, False, False, True, True]
    )

    # Asignar máximo un curso por estudiante
    asignados = []
    asignados_ruts = set()
    vacantes_disponibles = df_catalogo.set_index('NRC')['Vacantes Ofrecidas Máximas'].to_dict()

    for _, row in df_demand_sorted.iterrows():
        rut = row['RUT UC']
        nrc = row['NRC']
        if rut in asignados_ruts:
            continue
        cupo = vacantes_disponibles.get(nrc, np.nan)
        if pd.isna(cupo) or cupo < 1:
            continue
        vacantes_disponibles[nrc] -= 1
        asignados.append(row)
        asignados_ruts.add(rut)

    df_asignados = pd.DataFrame(asignados)

    # Salir si no se asignó nadie
    if df_asignados.empty:
        print("No quedan asignaciones individuales posibles.")
        break

    # Actualizar catálogo restando vacantes
    for nrc, usados in df_asignados['NRC'].value_counts().items():
        df_catalogo.loc[df_catalogo['NRC'] == nrc, 'Vacantes Ofrecidas Máximas'] -= usados

    # Estudiantes no asignados
    ruts_asignados = set(df_asignados['RUT UC'])
    df_no_asignados = df_validados[~df_validados['RUT UC'].isin(ruts_asignados)][['Nombre completo', 'RUT UC']].drop_duplicates()

    # Guardar output individual
    output_name = f'output_single_{single_iter}.xlsx'
    with pd.ExcelWriter(output_name) as writer:
        df_asignados.to_excel(writer, sheet_name='Con éxito', index=False)
        df_no_asignados.to_excel(writer, sheet_name='Sin éxito', index=False)

    print(f"{len(df_asignados)} estudiantes asignados en {output_name}")

    with pd.ExcelWriter(f'formulario_single_{single_iter}.xlsx', engine='openpyxl') as writer:
        df_validados[~df_validados['RUT UC'].isin(ruts_asignados)].to_excel(writer, sheet_name='Cursos validados', index=False)
        df_catalogo.to_excel(writer, sheet_name='Catálogo VRAI', index=False)
        df_convenios.to_excel(writer, sheet_name='Convenios', index=False)

    df_validados = df_validados[~df_validados['RUT UC'].isin(ruts_asignados)].copy()
    single_iter += 1

Iniciando asignación individual desde output_5.xlsx...
Iteración de asignación individual 1
119 estudiantes asignados en output_single_1.xlsx
Iteración de asignación individual 2
No quedan asignaciones individuales posibles.
